# Final Coursera Project - Melbourne Restaurant Analysis

### The following notebook contains my final project and report.

#### Introduction/Business Problem

I am thinking of opeining an Italian restaurant in Melbourne but am unsure where to open it. It would be best to be outside of suburbs with a lot of existing Italian restaurants but also not too far away that people would not think to look for an Italian restaurant in that location. My business problem is to find a suburb adjacent to suburbs with a strong Italian restaurant presence.

#### Data Requirements

This report will analyse the ditribution of restaurant types in Melbourne suburbs using foursquare data (venues/explore). I will find a list of Melbourne suburbs online and scrape the data. I will then use geopy.geocoders to find the latitude and longitude. I will find the number of Italian restaurants by suburb, and also the most common restaurant type by suburb and visualize the results using folium maps.

I have not completed the week 5 project yet so please disregard the code below for now. :-)

Firstly, import the required libraries

In [1]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [55]:
url = "https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs"

In [56]:
url_df = pd.read_html(url, header=0)

In [57]:
df = url_df[0]
df.head()

,Suburb,Postcode,Local government area,Location[citation needed],Distance[3][citation needed],Area[citation needed],Population[citation needed],Population density[citation needed],Date established[citation needed]
0,Bellfield,3081,City of Banyule,NaN,NaN,0.9 km2,"1,793[4]",NaN,NaN
1,Briar Hill,3088,City of Banyule,NaN,NaN,NaN,"3,152[4]",NaN,NaN
2,Bundoora,3083,City of Banyule; City of Darebin; City of Whit...,NaN,NaN,15 km2,28653,NaN,NaN
3,Eaglemont,3084,City of Banyule,NaN,NaN,1.9 km2,3873,NaN,NaN
4,Eltham,3095,City of Banyule; Shire of Nillumbik,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df['Address'] = df['Suburb'] + ', VIC ' + df['Postcode'].apply(str) + ", Australia"

In [59]:
geolocator = Nominatim(user_agent="specify_your_app_name_here")

In [60]:
def geoConvLat(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.latitude
    else: return 0
def geoConvLong(address):
    a = geolocator.geocode(address, timeout=15)
    if a != None:
        return a.longitude
    else: return 0

In [61]:
#lats = []
#for i in df['Address']:
#    lats.append(geoConvLat(i))

In [62]:
#longs = []
#for i in df['Address']:
#    longs.append(geoConvLong(i))

In [63]:
#lats = pd.Series(lats)
#longs = pd.Series(longs)
#df['Latitude'] = lats
#df['Longitude'] = longs

In [64]:
#df.to_csv('suburbs_and_coords.csv')

# Load Suburbs with Coordinates

In [2]:
df = pd.read_csv('suburbs_and_coords.csv')

In [3]:
df[df['Suburb'] == "Woodstock"]

,Unnamed: 0,Suburb,Postcode,Local government area,Location[citation needed],Distance[3][citation needed],Area[citation needed],Population[citation needed],Population density[citation needed],Date established[citation needed],Address,Latitude,Longitude
464,464,Woodstock,3755,City of Whittlesea,NaN,NaN,NaN,NaN,NaN,NaN,"Woodstock, VIC 3755, Australia",-37.546049,145.054427


In [8]:
df.shape

(549, 13)

In [9]:
drop_columns = ['Population[citation needed]', 'Area[citation needed]', 'Population density[citation needed]', 
                'Unnamed: 0', 'Location[citation needed]', 'Distance[3][citation needed]', 'Date established[citation needed]',
               'Address']
df = df.drop(drop_columns, axis=1)

In [10]:
df.head()

,Suburb,Postcode,Local government area,Latitude,Longitude
0,Bellfield,3081,City of Banyule,-37.753107,145.038478
1,Briar Hill,3088,City of Banyule,-37.706370,145.121648
2,Bundoora,3083,City of Banyule; City of Darebin; City of Whit...,-37.697306,145.066254
3,Eaglemont,3084,City of Banyule,-37.765144,145.063331
4,Eltham,3095,City of Banyule; Shire of Nillumbik,-37.713387,145.147880


In [11]:
import folium

# create map using latitude and longitude values
map_melbourne = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# add markers to map
for lat, lng, suburb, lga in zip(df['Latitude'], df['Longitude'], df['Suburb'], df['Local government area']):
    label = '{}, {}'.format(suburb, lga)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.5,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

In [12]:
CLIENT_ID = 'LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML' # your Foursquare ID
CLIENT_SECRET = 'AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM' # your Foursquare Secret
VERSION = '20190701' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML
CLIENT_SECRET:AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM


In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
lat, long = (-37.9136, 144.9631)
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LH2UYK41GOP20PJ0EMGOGJJZTNDBZIRGQIRZCOUU1JLRGEML&client_secret=AIP5GKY0EUPAIZNTNEOD3XKQ34YBHQ151HI4XBLURWPVXKFM&v=20190701&ll=-37.9136,144.9631&radius=1000&limit=100'

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
melbourne_venues = getNearbyVenues(names=df['Suburb'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [16]:
melbourne_venues.groupby('Suburb')['Suburb'].count()
print("Done")

Done


In [17]:
print('There are {} uniques categories.'.format(len(melbourne_venues['Venue Category'].unique())))

There are 215 uniques categories.


In [18]:
# one hot encoding
onehot = pd.get_dummies(melbourne_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Suburb'] = melbourne_venues['Suburb'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Suburb,Adult Boutique,Afghan Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant
0,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bellfield,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
onehot.shape

(8037, 216)

In [20]:
grouped = onehot.groupby('Suburb').sum().reset_index()
grouped.head()

,Suburb,Adult Boutique,Afghan Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant
0,Abbotsford,0,0,0,0,0,0,0,1,0,...,0,1,1,0,1,0,0,0,0,1
1,Aberfeldie,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Aintree,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Airport West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Albanvale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [21]:
grouped.shape

(307, 216)

In [22]:
grouped.head()

,Suburb,Adult Boutique,Afghan Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant
0,Abbotsford,0,0,0,0,0,0,0,1,0,...,0,1,1,0,1,0,0,0,0,1
1,Aberfeldie,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Aintree,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Airport West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Albanvale,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [23]:
# set number of clusters
#kclusters = 10

#grouped_clustering = grouped.drop('Suburb', 1)

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_

In [39]:
merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(grouped.set_index('Suburb'), on='Suburb')

merged.tail()

,Suburb,Postcode,Local government area,Latitude,Longitude,Adult Boutique,Afghan Restaurant,Antique Shop,Aquarium,Arcade,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant
544,Woori Yallock,3139,Shire of Yarra Ranges,-37.779333,145.530126,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
545,Yarra Glen,3775,Shire of Yarra Ranges,-37.657348,145.374396,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
546,Yarra Junction,3797,Shire of Yarra Ranges,-37.782169,145.615026,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
547,Yellingbo,3139,Shire of Yarra Ranges,-37.813513,145.508205,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
548,Yering,3770,Shire of Yarra Ranges,-37.688713,145.374657,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
merged = merged.fillna(0)

In [41]:
merged.to_csv('mergedData.csv', index=None)

# Load Suburbs Merged with Venues

In [9]:
merged= pd.read_csv('mergedData.csv')
keepSuburbs = pd.read_csv('keepSuburbs.csv')

In [172]:
keepList = []
for i in keepSuburbs['Suburb']:
    keepList.append(i)

keepPostCode[:2]

['beaumaris', 'black rock']

In [15]:
merged = merged[merged['Suburb'].isin(keepList)]

In [16]:
merged['Italian Restaurant'].unique()

array([ 0.,  1.,  3.,  2., 14.,  5.,  4., 10.])

In [17]:
merged.shape

(78, 220)

In [18]:
house_prices = pd.read_excel('melbourneSalesPrices.xlsx', index=None)

In [19]:
house_prices.shape

(48427, 2)

In [20]:
houses_grouped = house_prices.groupby('Suburb').mean().reset_index()

In [21]:
houses_grouped.sort_values(by='Price')[:5]

,Suburb,Price
108,Darley,380000.000000
204,Kurunjang,381785.714286
224,Melton South,396267.500000
90,Cockatoo,397000.000000
240,Mount Dandenong,405000.000000


In [32]:
joined_df = merged.join(houses_grouped.set_index('Suburb'), on='Suburb', how='inner')

In [33]:
joined_df.head()

,Suburb,Postcode,Local government area,Latitude,Longitude,Adult Boutique,Afghan Restaurant,Antique Shop,Aquarium,Arcade,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant,Price
21,Beaumaris,3193,City of Bayside,-37.982212,145.038910,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.514327e+06
22,Black Rock,3193,City of Bayside,-37.970519,145.022952,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.541980e+06
23,Brighton,3186,City of Bayside,-37.905608,145.002751,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.027742e+06
24,Brighton East,3187,City of Bayside,-37.917173,145.016366,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.686790e+06
25,Cheltenham,3192,City of Bayside; City of Kingston,-37.963418,145.061567,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.691274e+05


In [34]:
# set number of clusters
kclusters = 5

merged_clustering = joined_df.drop(['Suburb', 'Postcode', 'Local government area', 'Latitude', 'Longitude'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42, n_init=100).fit(merged_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([3, 3, 0, 3, 1, 1, 1, 3, 1, 1, 1, 3, 3, 0, 3, 3, 0, 0, 3, 2, 2, 3,
       3, 2, 3, 2, 1, 4, 1, 1, 2, 4, 2, 4, 2, 1, 1, 2, 2, 4, 2, 4, 4, 4,
       2, 2, 4, 1, 1, 0, 4, 1, 0, 4, 4, 2, 3, 3, 0, 3, 1, 3, 1, 1, 1, 2,
       4, 1, 2, 2, 3, 1])

In [35]:
joined_df.insert(column ='Cluster', value = kmeans.labels_, loc = 1)

In [36]:
joined_df.head()

,Suburb,Cluster,Postcode,Local government area,Latitude,Longitude,Adult Boutique,Afghan Restaurant,Antique Shop,Aquarium,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant,Price
21,Beaumaris,3,3193,City of Bayside,-37.982212,145.038910,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.514327e+06
22,Black Rock,3,3193,City of Bayside,-37.970519,145.022952,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.541980e+06
23,Brighton,0,3186,City of Bayside,-37.905608,145.002751,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.027742e+06
24,Brighton East,3,3187,City of Bayside,-37.917173,145.016366,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.686790e+06
25,Cheltenham,1,3192,City of Bayside; City of Kingston,-37.963418,145.061567,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.691274e+05


In [37]:
# create map
map_clusters = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
import math
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, p, in zip(joined_df['Latitude'], joined_df['Longitude'], joined_df['Suburb'], 
                                     joined_df['Cluster'], joined_df['Price']):
    label = folium.Popup(str(poi) + ", " + str(cluster) + ", " + str(p//1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=1
    ).add_to(map_clusters)
    
       
map_clusters

In [183]:
import json
import geopandas as gdp

input_dict = gdp.read_file('https://data.gov.au/geoserver/vic-suburb-locality-boundaries-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_af33dd8c_0534_4e18_9245_fc64440f742e&outputFormat=json')

# Filter python objects with list comprehensions
print(type(input_dict))

# Transform python object back into json
#output_json = json.dumps(output_dict)

# Show json
#print output_json

<class 'geopandas.geodataframe.GeoDataFrame'>


In [184]:
input_dict.head(2)

,id,lc_ply_pid,dt_create,dt_retire,loc_pid,vic_locali,vic_loca_1,vic_loca_2,vic_loca_3,vic_loca_4,vic_loca_5,vic_loca_6,vic_loca_7,geometry
0,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.1,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"(POLYGON ((141.74552399 -35.07228701, 141.7455..."
1,ckan_af33dd8c_0534_4e18_9245_fc64440f742e.2,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"(POLYGON ((148.668767 -37.39571245, 148.668762..."


In [185]:
vic_loca_2 = input_dict['vic_loca_2'].apply(lambda x: x.lower())

In [186]:
input_dict['vic_loca_2'] = vic_loca_2

In [220]:
keepPostCode = []
for i in joined_df['Suburb']:
    keepPostCode.append(i.lower())

In [221]:
test_dict = input_dict[input_dict['vic_loca_2'].isin(keepPostCode)]

In [222]:
test_dict = test_dict[['vic_loca_2', 'geometry']] 

In [223]:
joined_df['lowerSuburb'] = joined_df['Suburb'].apply(lambda x: x.lower())

In [224]:
joined_df.head().head()

,Suburb,Cluster,Postcode,Local government area,Latitude,Longitude,Adult Boutique,Afghan Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yunnan Restaurant,Price,lowerSuburb
21,Beaumaris,3,3193,City of Bayside,-37.982212,145.038910,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.514327e+06,beaumaris
22,Black Rock,3,3193,City of Bayside,-37.970519,145.022952,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.541980e+06,black rock
23,Brighton,0,3186,City of Bayside,-37.905608,145.002751,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.027742e+06,brighton
24,Brighton East,3,3187,City of Bayside,-37.917173,145.016366,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.686790e+06,brighton east
25,Cheltenham,1,3192,City of Bayside; City of Kingston,-37.963418,145.061567,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.691274e+05,cheltenham


In [225]:
graph_df = joined_df[['lowerSuburb', 'Price']]

In [226]:
test_dict.shape

(71, 2)

In [227]:
map_clusters = folium.Map(location=(-37.9136, 144.9631), zoom_start=9)
map_clusters.choropleth(
geo_data=test_dict,
data = graph_df,
key_on='properties.vic_loca_2',
fill_color='red', #Colors to fill the choropleth
)
map_clusters

C:\Users\Matt Burt\Anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


ValueError: red is not a valid ColorBrewer code

In [228]:
graph_df.shape, test_dict.shape

((72, 2), (71, 2))